In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import palettable.colorbrewer.sequential as pcs
import palettable.colorbrewer.diverging as pcd

from statsmodels.stats.outliers_influence import variance_inflation_factor

import libpysal

from esda.moran import Moran,Moran_Local

from spreg import OLS, ML_Lag

from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR

from splot.esda import lisa_cluster

from patsy import dmatrices

In [ ]:
# read in data
pre_df=pd.read_csv('data/lsoa_data.csv')
# lsoa geometry
boundary=gpd.read_file('data/LSOA_boundary/LDN-LSOAs.shp')

# merge two dataframes
gdf=gpd.GeoDataFrame(pre_df).merge(boundary,left_on='Codes',right_on='lsoa11cd')
# drop excessive columns
gdf=gdf.drop(['lsoa11cd','lsoa11nm','lsoa11nmw','objectid','st_areasha','st_lengths'],axis=1)

In [ ]:
# explore the structure of the data
gdf['kMedianHP']=gdf['MedianHP']/1000
gdf['kMedianHP'].describe()

In [ ]:
# remove lsoas wth no house price data
gdf=gdf[gdf['kMedianHP']>0]

# check correlation between dependent and independent variables
gdf.corr(method='kendall')['MedianHP']

In [ ]:
gdf.plot(column='kMedianHP',scheme='fisherjenks',cmap='viridis',legend=True)

In [ ]:
gdf[gdf['kMedianHP']>1500]

In [ ]:
# list all the independent variables
var_n=gdf.columns.tolist()[3:-2]

In [ ]:
var_n.remove('c_per_hhlds')
var_n.remove('Pct_CHDC')

gdf.drop(['Pct_CHDC','c_per_hhlds'],inplace=True,axis=1)

In [ ]:
def vif(df,dep_var,list):
    form=''
    for i in list:
        form+=i
        if i != list[-1]:
            form+='+'
    y,X=dmatrices(dep_var+ ' ~ ' + form, data=df,return_type='dataframe')

    vif=pd.DataFrame()
    vif['VIF']=[variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['variable']=X.columns

    return vif

In [ ]:
# check VIF
vif(gdf,'kMedianHP',var_n)

In [ ]:
# Check correlation with MedianIncome
gdf.corr(method='kendall')['MedianIncome']

In [ ]:
gdf.corr(method='kendall')['Pct_qual_above_l4']

In [ ]:
var_n.remove('UnRate')
var_n.remove('Pct_workingage')

gdf.drop(['UnRate','Pct_workingage'],inplace=True,axis=1)

In [ ]:
# recheck VIF
vif(gdf,'kMedianHP',var_n)

In [ ]:
# build simple OLS model
m_multi=OLS(gdf[['kMedianHP']].values,
            gdf[var_n].values,
            name_x=var_n,
            name_y='kMedianHP')

print(m_multi.summary)

In [ ]:
# spatial weights matrix
weights=libpysal.weights.KNN.from_dataframe(gdf,k=6)
weights.transform='r'

In [ ]:
# check the normality of the residuals
plt.plot(m_multi.predy,
         m_multi.u,
         'oC0',
         markeredgecolor='white')
plt.axhline(linestyle='dashed',color='red')
plt.ylabel('Residuals')
plt.xlabel('Predicted y')
plt.show()

In [ ]:
# spatial autocorrelation of the OLS residuals
gdf['multi_res']=m_multi.u
multi_res_moran=Moran(m_multi.u,weights)
print(multi_res_moran.I)
print(multi_res_moran.p_sim)

In [ ]:
fig,ax=plt.subplots(1,figsize=(15,12))

gdf.plot(column='multi_res',scheme='FisherJenks',k=7,ax=ax,
         cmap=pcs.YlGnBu_7.mpl_colormap,edgecolor='lightgrey',linewidth=0.1,
         legend=True,legend_kwds={'loc':'lower right','title':'Residuals'})

plt.figtext(x=0.35,y=0.13,
            s="Global Moran's I: "+str(round(multi_res_moran.I,3))+" (p-value < "+str(multi_res_moran.p_sim)+")",fontsize=16)
ax.set_axis_off()

#plt.savefig('graph/map_res.png',dpi=200, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# build spatial lag model
m_Lag=ML_Lag(gdf[['kMedianHP']].values,
                   gdf[var_n].values,
                   weights,
                   name_x=var_n,
                   name_y='kMedianHP')
print(m_Lag.summary)

In [ ]:
#Prepare dataset inputs
g_y = gdf['kMedianHP'].values.reshape((-1,1))
g_X = gdf[var_n].values
g_coords=[]
for i in gdf['geometry'].centroid.tolist():
    g_coords.append((i.x,i.y))

In [ ]:
bw=Sel_BW(g_coords,g_y,g_X,fixed=False,kernel='gaussian',spherical=False)
bw.search()

In [ ]:
# build GWR model
m_gwr=GWR(g_coords,g_y,g_X,bw.bw[0],kernel='gaussian')
m_gwr_fit=m_gwr.fit()
m_gwr_fit.summary()

In [ ]:
# filter all the statistically insignificant (at 0.05 level) results
f_est=m_gwr_fit.filter_tvals(alpha=.05)

In [ ]:
var_n

In [ ]:
# merge the model result with the original dataframe
data_params=pd.DataFrame(f_est)
data_localR2=pd.DataFrame(m_gwr_fit.localR2)

tem_df=pd.DataFrame(gdf[['Codes','Names','kMedianHP','geometry']])
result_df=tem_df.assign(intercept=data_params[0],
                        MedianIncome=data_params[1],
                        Pct_nonwhite=data_params[2],
                        Pct_workingage=data_params[3],
                        UnRate=data_params[4],
                        PTAL_average=data_params[5],
                        localR2=data_localR2[0])

result_gdf=gpd.GeoDataFrame(result_df,geometry='geometry')

In [ ]:
fig,ax=plt.subplots(2,4,figsize=(20,13), subplot_kw=dict(aspect='equal'))
ax = ax.flatten()

titles = ['Intercept']+var_n

for i,row in enumerate(f_est.T):
    temp = result_gdf.assign(toplot=f_est.T[i])
    temp.query('toplot==0').sort_values('toplot').plot(color='grey',ax=ax[i],alpha=.2)

    temp.query('toplot!=0').sort_values('toplot').plot('toplot',cmap='viridis',ax=ax[i],legend=True)
    
    ax[i].set_title(titles[i], fontsize=16)
    
    ax[i].set_xticklabels([])
    ax[i].set_yticklabels([])
    ax[i].set_xticks([])
    ax[i].set_yticks([])

result_gdf.assign(r2=m_gwr_fit.localR2).sort_values('r2').plot('r2',ax=ax[-1],legend=True,
                                                               vmin=0,vmax=1,cmap='viridis')
    
ax[-1].set_xticklabels([])
ax[-1].set_yticklabels([])
ax[-1].set_xticks([])
ax[-1].set_yticks([])
    
ax[-1].set_title('Local R2', fontsize=16)
    
fig.tight_layout()
    
plt.show()

In [ ]:
# calculate residuals
result_gdf['predy']=m_gwr_fit.predy
result_gdf['resid']=result_gdf['kMedianHP']-result_gdf['predy']
result_gdf['pct_resid']=result_gdf['resid']/result_gdf['kMedianHP']

In [ ]:
# plot the residuals spatial distribution
fig,ax=plt.subplots(1,figsize=(12,8))
result_gdf.plot(column='pct_resid',cmap=pcd.PRGn_7.mpl_colormap,scheme='fisherjenks',k=7,legend=True,ax=ax)
plt.show()

In [ ]:
# global moran's I of the GWR residuals
gwr_res_moran=Moran(result_gdf['pct_resid'].values,weights)
print(gwr_res_moran.I)
print(gwr_res_moran.p_sim)

In [ ]:
# local moran's I
moran_loc_res=Moran_Local(result_gdf['pct_resid'].values,weights,permutations=999)

In [ ]:
# lisa cluster map
fig,ax=plt.subplots(1,figsize=(15,12))
lisa_cluster(moran_loc_res,gdf,p=0.05,ax=ax)
plt.show()